In [1]:
from data import *


In [11]:
import xarray
abs_path = "/proj/cvl/users/x_juska/repos/graphcast/"
nc_path =  abs_path +"data/22_3d_"
spath = nc_path + f"surface.nc"
org_surface = xarray.open_dataset(spath).compute()

In [13]:
surface = org_surface
surface = surface.rename(
    {var: singlelevelfields[i] for i, var in enumerate(surface.data_vars)}
)
surface = surface.rename({"geopotential": "geopotential_at_surface"})

# Rename axes
surface = surface.rename(
    {"valid_time": "time", "latitude": "lat", "longitude": "lon"}
)

# Calculating the sum of the last 6 hours of rainfall
surface = surface.sortby("time")
surface["total_precipitation_6hr"] = (
    surface["total_precipitation"].rolling(time=6).sum()
)
surface = surface.drop_vars("total_precipitation")
surface = trim_time(surface)
surface

<xarray.Dataset> Size: 365MB
Dimensions:                       (time: 11, lat: 721, lon: 1440)
Coordinates:
    number                        int64 8B 0
  * time                          (time) datetime64[ns] 88B 2022-01-01T06:00:...
  * lat                           (lat) float64 6kB 90.0 89.75 ... -89.75 -90.0
  * lon                           (lon) float64 12kB 0.0 0.25 ... 359.5 359.8
    expver                        (time) <U4 176B '0001' '0001' ... '0001'
Data variables:
    10m_u_component_of_wind       (time, lat, lon) float32 46MB -0.259 ... 0....
    10m_v_component_of_wind       (time, lat, lon) float32 46MB 0.08087 ... 0...
    2m_temperature                (time, lat, lon) float32 46MB 248.1 ... 246.6
    geopotential_at_surface       (time, lat, lon) float32 46MB 0.3228 ... 2....
    land_sea_mask                 (time, lat, lon) float32 46MB 0.0 0.0 ... 1.0
    mean_sea_level_pressure       (time, lat, lon) float32 46MB 1.031e+05 ......
    toa_incident_solar_radiation  (time, lat, lon) float32 46MB 0.0 ... 1.961...
    total_precipitation_6hr       (time, lat, lon) float32 46MB 5.245e-06 ......
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-09-14T09:30 GRIB to CDM+CF via cfgrib-0.9.1...

In [14]:
apath = nc_path + f"atmo.nc"
org_atmo = xarray.open_dataset(apath).compute()

In [16]:
atmo = org_atmo
atmo = atmo.rename(
    {var: pressurelevelfields[i] for i, var in enumerate(atmo.data_vars)}
)

# Rename axes for pressurelevel as well
atmo = atmo.rename(
    {
        "valid_time": "time",
        "latitude": "lat",
        "longitude": "lon",
        "pressure_level": "level",
    }
)
atmo = atmo.isel(time=slice(1, None))
atmo

<xarray.Dataset> Size: 10GB
Dimensions:              (time: 11, level: 37, lat: 721, lon: 1440)
Coordinates:
    number               int64 8B 0
  * time                 (time) datetime64[ns] 88B 2022-01-01T06:00:00 ... 20...
  * level                (level) float64 296B 1e+03 975.0 950.0 ... 3.0 2.0 1.0
  * lat                  (lat) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * lon                  (lon) float64 12kB 0.0 0.25 0.5 ... 359.2 359.5 359.8
    expver               (time) <U4 176B '0001' '0001' '0001' ... '0001' '0001'
Data variables:
    u_component_of_wind  (time, level, lat, lon) float32 2GB 1.526e-05 ... -0...
    v_component_of_wind  (time, level, lat, lon) float32 2GB -0.0002899 ... 0...
    geopotential         (time, level, lat, lon) float32 2GB 2.177e+03 ... 4....
    specific_humidity    (time, level, lat, lon) float32 2GB 0.0004974 ... 3....
    temperature          (time, level, lat, lon) float32 2GB 251.2 ... 285.9
    vertical_velocity    (time, level, lat, lon) float32 2GB 0.02624 ... 0.00...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-09-14T09:33 GRIB to CDM+CF via cfgrib-0.9.1...

In [18]:
from matplotlib.pyplot import pie


data = xarray.merge([surface, atmo])
data = pipeline(data)
data

<xarray.Dataset> Size: 10GB
Dimensions:                       (time: 11, lat: 721, lon: 1440, level: 37,
                                   batch: 1)
Coordinates:
  * time                          (time) timedelta64[ns] 88B 0 days 00:00:00 ...
  * lat                           (lat) float64 6kB -90.0 -89.75 ... 89.75 90.0
  * lon                           (lon) float64 12kB 0.0 0.25 ... 359.5 359.8
  * level                         (level) float64 296B 1.0 2.0 ... 975.0 1e+03
    datetime                      (batch, time) datetime64[ns] 88B 2022-01-01...
Dimensions without coordinates: batch
Data variables: (12/14)
    10m_u_component_of_wind       (batch, time, lat, lon) float32 46MB 0.5554...
    10m_v_component_of_wind       (batch, time, lat, lon) float32 46MB 0.0398...
    2m_temperature                (batch, time, lat, lon) float32 46MB 249.0 ...
    geopotential_at_surface       (batch, lat, lon) float32 4MB 2.736e+04 ......
    land_sea_mask                 (batch, lat, lon) float32 4MB 1.0 1.0 ... 0.0
    mean_sea_level_pressure       (batch, time, lat, lon) float32 46MB 9.957e...
    ...                            ...
    u_component_of_wind           (batch, time, level, lat, lon) float32 2GB ...
    v_component_of_wind           (batch, time, level, lat, lon) float32 2GB ...
    geopotential                  (batch, time, level, lat, lon) float32 2GB ...
    specific_humidity             (batch, time, level, lat, lon) float32 2GB ...
    temperature                   (batch, time, level, lat, lon) float32 2GB ...
    vertical_velocity             (batch, time, level, lat, lon) float32 2GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-09-14T09:30 GRIB to CDM+CF via cfgrib-0.9.1...

: 